# 키버트(KeyBERT)를 이용한 키워드 추출

단순 통계적 특성 뿐만아니라 의미적 유사성을 고려하기 위해서 여기서는 SBERT 임베딩을 활용하여 사용하기 쉬운 키워드 추출 알고리즘인 KeyBERT를 사용합니다.

In [1]:
%pip install keybert sentence_transformers

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pytorch-lightning 1.6.1 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
import os
cwd = os.getcwd()

try :
    os.mkdir(cwd + "/models") 
except:
    pass

In [ ]:
os.chdir(cwd + "/models")

try :
    os.system("git clone https://huggingface.co/lcw99/t5-base-korean-text-summary")
except:
    print('이미 모델이 존재합니다.')

os.chdir(cwd)

KeyBert[https://github.com/MaartenGr/KeyBERT#embeddings] 모듈을 사용하여 키워드를 쉽게 추출할 수 있습니다.   

In [2]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer(cwd + '/models/paraphrase-multilingual-MiniLM-L12-v2', device=device)
kw_model = KeyBERT(model)

In [3]:
text = """
서울 지진 대피소에 대한 데이터 분석을 위해서는 어떤 종류의 데이터가 필요할까요? 예를 들어, 서울시의 지진 대피소 위치, 수용 가능 인원, 대피소 내부 시설물, 대피소 이용 현황 등의 정보가 필요할 것입니다. 
지진 대피소 위치 분석 예시: 지진 대피소 위치는 서울시 공공데이터 포털에서 제공하는 "서울시 지진대피소 안내" 데이터를 사용할 수 있습니다. 이 데이터셋에는 지진 대피소 명칭, 위치(주소), 좌표, 수용 인원, 관리 기관 등의 항목이 포함되어 있습니다. 
이를 바탕으로 대피소 위치를 지도에 시각화하여 지진 발생 시 대피소가 필요한 지역을 파악할 수 있습니다. 대피소 이용 현황 분석 예시: 대피소 이용 현황은 서울시에서 제공하는 "서울시 재난정보 실시간 수집 및 제공 서비스" 데이터를 사용할 수 있습니다. 
이 데이터셋에는 대피소 이용 현황(대피소 이용 가능 여부, 이용 중인 인원 수), 지진 발생 시 대피소 이용 현황 등의 정보가 포함되어 있습니다. 이를 바탕으로 대피소 이용 현황을 분석하여 인원이 많은 대피소를 파악하거나, 대피소 이용 가능 여부 등을 파악할 수 있습니다.
"""

In [16]:
keywords = kw_model.extract_keywords(text)
print(keywords)

keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(2, 2))
print(keywords)

[('지진대피소', 0.4972), ('지진', 0.4423), ('공공데이터', 0.4249), ('서울', 0.4239), ('서울시에서', 0.3922)]
[('서울시 지진대피소', 0.6382), ('서울시의 지진', 0.6199), ('서울 지진', 0.6039), ('데이터셋에는 지진', 0.5942), ('서울시 공공데이터', 0.561)]


5개의 키워드가 출력되는데, 몇몇 키워드는 의미가 좀 비슷해보입니다.  
비슷한 의미의 키워드들이 반환되는 데는 이 키워드들이 문서를 가장 잘 나타내고 있기 때문입니다.  
다양한 키워드를 출력하고 싶다면 선정의 정확성과 다양성 사이의 조절이 필요합니다.

KeyBert 모듈은 다음과 같은 연관 키워드 조합 관련 알고리즘을 지원합니다.
- Max Sum Similarity
- Maximal Marginal Relevance

## Max Sum Similarity
 
Max Sum Similarity 알고리즘은 후보(candidate)들 간의 유사성을 최소화 하면서 원문과 후보간 유사성을 극대화합니다.   
즉, 코사인 유사도에 기반하여 원문과 유사한 상위 nr_candidates 개의 단어를 선정하고  
단어간 덜 유사한 키워드 간 조합을 계산합니다.   
낮은 nr_candidates를 설정하면 결과는 출력된 키워드 5개는 유사하며,  
상대적으로 높은 nr_candidates는 더 다양한 키워드 5개를 만듭니다.

In [14]:
kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1),
                              use_maxsum=True, nr_candidates=30, top_n=5)

[('예를', 0.1343),
 ('시설물', 0.2401),
 ('서울', 0.4239),
 ('공공데이터', 0.4249),
 ('지진', 0.4423)]

In [13]:
kw_model.extract_keywords(text, keyphrase_ngram_range=(2, 2),
                              use_maxsum=True, nr_candidates=30, top_n=5)

[('실시간 수집', 0.2795),
 ('위치 주소', 0.3053),
 ('현황은 서울시에서', 0.3489),
 ('데이터 분석을', 0.3763),
 ('인원 지진', 0.4429)]

## MMR(Maximal Marginal Relevance)

문서와 코사인 유사도가 가장 유사한 키워드를 선택한 후,   
문서와는 유사하나 이미 선택된 키워드와 유사하지 않은 새로운 후보를 반복적으로 선택하는 알고리즘 입니다.

diversity 변수는 0~1사이의 소수이며,   
낮은 diversity 값을 설정한다면, 결과는 기존의 코사인 유사도만 사용한 것과 매우 유사하고,   
상대적으로 높은 diversity값은 더 다양한 키워드를 추출합니다.


In [ ]:
kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1),
                              use_mmr=True, diversity=0.5)

[('지진대피소', 0.4972),
 ('공공데이터', 0.4249),
 ('서울', 0.4239),
 ('재난정보', 0.3739),
 ('시설물', 0.2401)]

In [ ]:
kw_model.extract_keywords(text, keyphrase_ngram_range=(2, 2),
                              use_mmr=True, diversity=0.5)

[('서울시 지진대피소', 0.6382),
 ('서울시 공공데이터', 0.561),
 ('재난정보 실시간', 0.4145),
 ('종류의 데이터가', 0.4055),
 ('위치 주소', 0.3053)]